# Automated hyperparameter tunning
To run this code you should have scikit-learn verstion 0.21.2
FIrst we import basic mobules we will need.

In [1]:
import tensorflow as tf

import sys
assert sys.version_info >= (3,5)

import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
import pandas as pd

# To plot
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
# for random search cv
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal
from sklearn.pipeline import Pipeline


In [2]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))
# it should be 0.21.2 because the hyper param tuning will not work

The scikit-learn version is 0.21.2.


# Here we import the data, the first comments is an example of how we import the data on mac the second on windows.

In [3]:
# "/Users/vasileioskilis/Desktop/projectA1988/dataset/GOOGL5yearsd.csv"
# "C:\\Users\\KILDE\\Desktop\\projectA\\dataset\\GOOGL5yearsd.csv"
stock=pd.read_csv("C:\\Users\\KILDE\\Desktop\\projectA\\dataset\\GOOGL5yearsd.csv")
stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-07-01,543.659973,545.809998,539.760010,543.299988,543.299988,1538000
1,2015-07-02,544.739990,548.000000,543.570007,547.340027,547.340027,1406200
2,2015-07-06,542.250000,548.580017,542.099976,545.619995,545.619995,1280700
3,2015-07-07,547.429993,551.000000,539.849976,550.030029,550.030029,1678900
4,2015-07-08,545.599976,548.280029,541.200012,541.700012,541.700012,1383100


# We create a panada series, with Date as an index to the closing price and scale the data

In [4]:
close_price = stock['Close']
close_price.index = stock['Date']
close_price.head()

Date
2015-07-01    543.299988
2015-07-02    547.340027
2015-07-06    545.619995
2015-07-07    550.030029
2015-07-08    541.700012
Name: Close, dtype: float64

In [5]:
close_price = stock.Close.values

In [6]:
close_price = np.reshape(close_price,(-1,1))

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_normalised = scaler.fit_transform(close_price)
data_normalised

array([[0.00162736],
       [0.00573656],
       [0.00398709],
       ...,
       [0.83489126],
       [0.87011407],
       [0.8913515 ]])

# We split the dataset into 80% train, 10% validation and 10% test

In [8]:
train_size = int(len(data_normalised) * 0.80)
half_rest = int((len(data_normalised) - train_size)/2)

train_set = data_normalised[0:train_size,:]
val_set = data_normalised[train_size:train_size+half_rest,:]
test_set = data_normalised[train_size+half_rest:len(data_normalised),:]

# Method that creates sliding windows

In [9]:
def createWindows(dataset, window):
    X_train = []
    Y_train = []
    
    for i in range(len(dataset)-window-1):
        X_train.append(dataset[i:(i+window),0])
        Y_train.append(dataset[i+window,0])
    return np.array(X_train),np.array(Y_train)

# Create the windows and reshape the data so we can feed them to the LSTM

Input shape for an lstm (batch_size, time_steps, seq_len), window size 20 ,single step prediction the last element of the window is the target

In [10]:
X_train, Y_train = createWindows(train_set,20)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

In [11]:
X_val, Y_val = createWindows(val_set,20)
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))

In [12]:
X_test, Y_test = createWindows(test_set,20)
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Here we do the walk forward validation split

In [13]:
from sklearn.model_selection import TimeSeriesSplit
dev_size=0.1 
n_splits=int((1//dev_size)-1)
tscv=TimeSeriesSplit(n_splits=n_splits) 

# This is the method that created our LSTM models

In [14]:
def createLSTM(n_hidden=2, n_neurons=200, learning_rate=0.001, input_shape=X_train.shape[-2:]):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(tf.keras.layers.LSTM(n_neurons, activation="relu",return_sequences=True))
        model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(1))
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

# Here we use a Keras regressor wrapper on our method that created the LSTM models

In [15]:
test_model= tf.keras.wrappers.scikit_learn.KerasRegressor(createLSTM)

In [16]:
test_model.fit(X_train, Y_train, epochs=15,
validation_data=(X_val, Y_val),
callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])
mse_test = test_model.score(X_test, Y_test)

Train on 986 samples, validate on 105 samples
Epoch 1/15
986/986 [==============================] - 4s 4ms/sample - loss: 0.0734 - val_loss: 0.0023
Epoch 2/15
986/986 [==============================] - 0s 183us/sample - loss: 0.0038 - val_loss: 9.8294e-04
Epoch 3/15
986/986 [==============================] - 0s 174us/sample - loss: 0.0028 - val_loss: 0.0015
Epoch 4/15
986/986 [==============================] - 0s 177us/sample - loss: 0.0026 - val_loss: 6.6029e-04
Epoch 5/15
986/986 [==============================] - 0s 184us/sample - loss: 0.0027 - val_loss: 6.7327e-04
Epoch 6/15
986/986 [==============================] - 0s 182us/sample - loss: 0.0025 - val_loss: 5.7515e-04
Epoch 7/15
986/986 [==============================] - 0s 184us/sample - loss: 0.0021 - val_loss: 6.1725e-04
Epoch 8/15
986/986 [==============================] - 0s 178us/sample - loss: 0.0025 - val_loss: 8.7115e-04
Epoch 9/15
986/986 [==============================] - 0s 197us/sample - loss: 0.0023 - val_loss: 4.6

# Here we do the randomized search
The cross validation methos (cv) is the walk forward validation. The scoring method is mean squared error and the parameters are the ones we wanted to check.

In [17]:
parameters = {
"n_hidden": [1,2,3],
"n_neurons": np.arange(30,250),
"learning_rate": [0.001,0.002,0.003],
}
searchCV = RandomizedSearchCV(test_model,parameters, n_iter=10,cv=tscv,scoring='neg_mean_squared_error',error_score=np.nan)
searchCV.fit(X_train, Y_train)

Train on 114 samples
114/114 [==============================] - 3s 26ms/sample - loss: 0.0243
Train on 223 samples
223/223 [==============================] - 3s 15ms/sample - loss: 0.0189
Train on 332 samples
332/332 [==============================] - 3s 10ms/sample - loss: 0.0173
Train on 441 samples
441/441 [==============================] - 3s 7ms/sample - loss: 0.0194
Train on 550 samples
550/550 [==============================] - 4s 7ms/sample - loss: 0.0217
Train on 659 samples
659/659 [==============================] - 3s 5ms/sample - loss: 0.0312
Train on 768 samples
768/768 [==============================] - 3s 4ms/sample - loss: 0.0347
Train on 877 samples
877/877 [==============================] - 3s 4ms/sample - loss: 0.0284
Train on 114 samples
114/114 [==============================] - 5s 46ms/sample - loss: 0.0294
Train on 223 samples
223/223 [==============================] - 5s 24ms/sample - loss: 0.0304
Train on 332 samples
332/332 [==============================] - 5

RandomizedSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=8),
                   error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x000001F1BA5DE688>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': [0.001, 0.002, 0.003],
                                        'n_hidden': [1, 2, 3],
                                        'n_neurons': array([ 30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  4...
       186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198,
       199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211,
       212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224,
       225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
       238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
             

# we best model and the bast score

In [18]:
searchCV.best_params_

{'n_neurons': 241, 'n_hidden': 1, 'learning_rate': 0.003}

In [19]:
searchCV.best_score_

-0.00515785121085937

# websites used to create this program

https://keras.io/